In [1]:
import os
import pandas as pd
from placekey.api import PlacekeyAPI
import json
import polars as pl
import pyarrow as pa

In [2]:
# Configuration
PLACEKEY_API_KEY = os.environ.get('PLACEKEY_API_KEY')  # Store API key in environment variable
if not PLACEKEY_API_KEY:
    raise ValueError("Please set the PLACEKEY_API_KEY environment variable.")

DATA_DIR = 'data'

In [3]:
def prep_df_for_api(df, column_map):
    """Prepares a DataFrame for the Placekey API.

    Args:
        df (pd.DataFrame): Input DataFrame.
        column_map (dict): Mapping of original column names to Placekey API field names.

    Returns:
        pd.DataFrame: DataFrame ready for Placekey API input.
    """

    df = df.rename(columns=column_map)[list(column_map.values())]
    df["iso_country_code"] = "US"
    df["query_id"] = df["query_id"].astype(str)
    return df


def add_placekeys_to_df(df, column_map, join_key, output_filename):
    """Processes a dataset through the Placekey API.

    Args:
        df (pd.DataFrame): Input DataFrame.
        column_map (dict): Mapping of column names.
        output_filename (str): Name of the CSV file to store Placekeys.

    Returns:
        pd.DataFrame: DataFrame with Placekeys.
    """
    df_for_api = prep_df_for_api(df, column_map)
    responses = PlacekeyAPI(PLACEKEY_API_KEY).lookup_placekeys(json.loads(df_for_api.to_json(orient="records")), verbose=True)
    placekeys_df = pd.DataFrame(responses)
    placekeys_df.to_csv(os.path.join(DATA_DIR, output_filename), index=False)
    return df.merge(placekeys_df, left_on=join_key, right_on="query_id", how="left")

In [ ]:
# Process Blight Violations
blight_df = pd.read_csv(os.path.join(DATA_DIR, 'Blight_Violations.csv'))
blight_df = blight_df.head(100)
blight_df['zip_code'] = blight_df['zip_code'].astype(str)
blight_df['state'] = blight_df['state'].astype(str)
blight_df['ticket_id'] = blight_df['ticket_id'].astype(str)
blight_column_map = {
    "ticket_id": "query_id",
    "violation_address" : "street_address",
    "state": "region",
    "zip_code": "postal_code",
    "Y" : "latitude",
    "X" : "longitude",
    "country" : "iso_country_code",
    "city" : "city"
    }
blight_df = add_placekeys_to_df(blight_df, blight_column_map, "ticket_id", "placekeys_Blight_Violations.csv")
blight_df.to_csv(os.path.join(DATA_DIR, "Blight_Violations_w_placekeys.csv"), index=False)

# Process Property Sales
sales_df = pd.read_csv(os.path.join(DATA_DIR, 'Property_Sales.csv'))
sales_df['sale_id'] = sales_df['sale_id'].astype(str)
sales_df['city'] = 'Detroit'
sales_df['iso_country_code'] = 'US'
sales_df['region'] = 'MI'
Property_Sales_column_map = {
    "sale_id": "query_id",
    "address" : "street_address",
    "Y" : "latitude",
    "X" : "longitude",
    "city": "city",
    "region": "region",
    "iso_country_code": "iso_country_code"
    }
sales_df = add_placekeys_to_df(sales_df, Property_Sales_column_map, "sale_id", "placekeys_Property_Sales.csv")
sales_df.to_csv(os.path.join(DATA_DIR, "Property_Sales_w_placekeys.csv"), index=False)

In [12]:
blight_df = pl.read_csv(os.path.join(DATA_DIR, "Blight_Violations_w_placekeys.csv"),infer_schema_length=1000000)
blight_df_placekeys = blight_df["placekey"].to_list()
blight_df = blight_df.filter(pl.col('placekey').is_not_null())

sales_df = pl.read_csv(os.path.join(DATA_DIR, "Property_Sales_w_placekeys.csv"),infer_schema_length=1000000)
sales_df = sales_df.filter(pl.col('placekey').is_not_null())

blight_df_joined_sales_df_placeykey = blight_df.join(sales_df, on="placekey", how="inner")
blight_df_joined_sales_df_placeykey.write_csv(os.path.join(DATA_DIR, "blight_joined_sales_placeykey.csv"))

In [8]:
sales_placekeys = pd.read_csv(os.path.join(DATA_DIR, "placekeys_Property_Sales.csv"))
blight_placekeys = pd.read_csv(os.path.join(DATA_DIR, "placekeys_Blight_Violations.csv"))
sales_placekeys.dropna(subset=['placekey'], inplace=True)
blight_placekeys.dropna(subset=['placekey'], inplace=True)

# Matching placekeys
matching_placekeys = sales_placekeys["placekey"].isin(blight_placekeys["placekey"])
print("Number of matching placekeys: {:,}".format(matching_placekeys.sum()))
print("Percentage of matching placekeys: {:.2f}%".format((matching_placekeys.sum() / len(sales_placekeys)) * 100))

# Same hex match
sales_placekeys['hex'] = sales_placekeys['placekey'].str.split('@').str[1]
blight_placekeys['hex'] = blight_placekeys['placekey'].str.split('@').str[1]
matching_hex = sales_placekeys["hex"].isin(blight_placekeys["hex"])
print("Number of matching hex: {:,}".format(matching_hex.sum()))
print("Percentage of matching hex: {:.2f}%".format((matching_hex.sum() / len(sales_placekeys)) * 100))


Number of matching placekeys: 142,295
Percentage of matching placekeys: 33.12%
Number of matching hex: 418,265
Percentage of matching hex: 97.35%
